# Binary Classifier: Waffles vs Pancakes

Run the code below to download the dataset `waffle_pancakes.zip`.

In [81]:
!gdown --id 1tJqtoF-5MNdC90rXYgtsXTs9o2odzEg5

Downloading...
From: https://drive.google.com/uc?id=1tJqtoF-5MNdC90rXYgtsXTs9o2odzEg5
To: d:\Github\waffles-vs-pancakes\waffle_pancakes.zip

0.00B [00:00, ?B/s]
524kB [00:00, 4.35MB/s]
1.57MB [00:00, 6.56MB/s]
2.62MB [00:00, 7.56MB/s]
3.67MB [00:00, 7.95MB/s]
5.24MB [00:00, 8.76MB/s]
6.82MB [00:00, 7.83MB/s]
8.39MB [00:01, 9.19MB/s]
9.96MB [00:01, 9.96MB/s]
11.0MB [00:01, 10.1MB/s]
12.6MB [00:01, 9.67MB/s]
13.0MB [00:01, 8.95MB/s]


In [82]:
import os
import zipfile

local_zip = './waffle_pancakes.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [83]:
# Directories with our training images
train_waf_dir = os.path.join('./waffle_pancakes/train/waffles')
train_pan_dir = os.path.join('./waffle_pancakes/train/pancakes')

# Directories with our test images
test_waf_dir = os.path.join('./waffle_pancakes/test/waffles')
test_pan_dir = os.path.join('./waffle_pancakes/test/pancakes')

In [84]:
train_waf_names = os.listdir(train_waf_dir)
print(train_waf_names[:10])
train_pan_names = os.listdir(train_pan_dir)
print(train_pan_names[:10])

test_waf_names = os.listdir(test_waf_dir)
test_pan_names = os.listdir(test_pan_dir)

['img-waf-0.png', 'img-waf-1.png', 'img-waf-10.png', 'img-waf-100.png', 'img-waf-101.png', 'img-waf-102.png', 'img-waf-103.png', 'img-waf-104.png', 'img-waf-105.png', 'img-waf-106.png']
['img-pan-0.png', 'img-pan-1.png', 'img-pan-10.png', 'img-pan-100.png', 'img-pan-101.png', 'img-pan-102.png', 'img-pan-103.png', 'img-pan-104.png', 'img-pan-105.png', 'img-pan-106.png']


In [86]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [87]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

In [88]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        './waffle_pancakes/train/',  # This is the source directory for training images
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary'
        )

validation_generator = validation_datagen.flow_from_directory(
        './waffle_pancakes/test/',  # This is the source directory for training images
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary'
        )

Found 947 images belonging to 2 classes.
Found 406 images belonging to 2 classes.


In [53]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/100
8/8 [==============================] - 39s 4s/step - loss: 0.6760 - accuracy: 0.5882 - val_loss: 1.1126 - val_accuracy: 0.0000e+00
Epoch 2/100
8/8 [==============================] - 32s 4s/step - loss: 0.6714 - accuracy: 0.6040
Epoch 3/100
8/8 [==============================] - 30s 3s/step - loss: 0.6663 - accuracy: 0.6040
Epoch 4/100
8/8 [==============================] - 30s 3s/step - loss: 0.6630 - accuracy: 0.6040
Epoch 5/100
2/8 [======>.......................] - ETA: 23s - loss: 0.6595 - accuracy: 0.6094

KeyboardInterrupt: 